In [1]:
from pymongo import MongoClient

In [2]:
c = MongoClient('10.0.1.40')

In [3]:
cur = c['corpora']['cnn'].find()
cur.next()
doc = cur.next()

In [4]:
doc['header']

"Senate Set for Nail-Biter Vote; Controversy Over Andy Puzder; Iran Reacts to Trump's Russia Comment; Al Qaeda Leader Comments on Trump; Pence Casts Tiebreaking Vote. Aired 12-12:30p ET"

In [5]:
import re

In [6]:
video_splitted = re.split(r'(\(BEGIN [A-Z ]*\)|\(END[A-Z ]*\))', doc['text'])

In [7]:
video_clips = []
normal_conv = [""]
i = 0
while i < len(video_splitted):
    if re.match(r'\(BEGIN [A-Z ]*\)', video_splitted[i]):
        video_clips.append(video_splitted[i+1])
        normal_conv.append("")
        i += 3
    else:
        normal_conv[-1] += video_splitted[i]
        i += 1

In [8]:
video_clips

["\n \nDONALD TRUMP, PRESIDENT OF THE UNITED STATES: We\\'re going to take it through the system. It\\'s very important. It\\'s very important for the country, regardless of me or whoever succeeds at a later date. I mean we have to have security in our country. We have to have the ability. \n \nWhen you take someplace like Syria, you take all of the different people going around -- and if you remember, ISIS said, we are going to infiltrate the United States and other countries through the migration. And then we\\'re not allowed to be tough on the people coming in? Explain that one. \n \n",
 " \n \nSEN. TIM KAINE (D), VIRGINIA: If you cannot be a champion for public schools, you should not be secretary of education.\n \nSEN. KAMALA HARRIS (D), CALIFORNIA: It is clear from her testimony, Betsy DeVos has not done her homework. \n \nSEN. BRIAN SCHATZ (D), HAWAII: We have the wrong person who may be confirmed as the secretary of education. \n \nSEN. ELIZABETH WARREN (D), MASSACHUSETTS: We n

In [9]:
documents = []
def process_group(group):
    doc = {}
    doc['convo'] = []
    turns = re.split(r'([\n\\n]*[ A-Z,-,{(}{)}]+[:])', group)
    i = 0
    while i < len(turns):
        if re.match(r'([\n\\n]*[ A-Z,-,{(}{)}]+[:])', turns[i]):
            speaker = clean_speaker(turns[i])
            if speaker.upper() == speaker:
                text = clean_text(turns[i+1])
                if text:
                    try:
                        sentences = cn.basic(text).json()['sentences']
                        endOfLast = 0
                        for sent in sentences:
                            last_word = sent['tokens'][-1]
                            sentence_text = text[endOfLast:last_word['characterOffsetEnd']+2]
                            endOfLast = last_word['characterOffsetEnd']+2
                            tokens = list(map(lambda x: x['word'], sent['tokens']))
                            doc['convo'].append({'speaker': speaker, 'text': sentence_text, 'tokens': tokens})
                    except Exception as e:
                        print(text)
            else:
                print(speaker)
                print(i)
            i += 2
        else:
            i += 1
    return doc

In [10]:
def process_doc(raw_doc):
    info = raw_doc['header']
    topics = info.split(';')
    topics[-1] = topics[-1].split("Aired")[0]
    show = raw_doc['show']
    documents = []
    video_splitted = re.split(r'(\(BEGIN [A-Z ]*\)|\(END[A-Z ]*\))', raw_doc['text'])
    video_clips = []
    normal_conv = [""]
    i = 0
    while i < len(video_splitted):
        if re.match(r'\(BEGIN [A-Z ]*\)', video_splitted[i]):
            video_clips.append(video_splitted[i+1])
            normal_conv.append("")
            i += 3
        else:
            normal_conv += video_splitted[i]
            i += 1
    for group in video_clips:
        doc = process_group(group)
        doc['topics'] = topics
        doc['show'] = show
        if len(doc['convo']) > 1:
            documents.append(doc)
    for group in normal_conv:
        doc = process_group(group)
        doc['topics'] = topics
        doc['show'] = show
        if len(doc['convo']) > 1:
            documents.append(doc)
    return documents

In [14]:
from tqdm import tqdm
documents = []
for doc in tqdm(c['corpora']['cnn'].find().limit(10), total=38000):
    documents.extend(process_doc(doc))

  0%|          | 10/38000 [00:01<1:27:02,  7.27it/s]


In [15]:
c['corpora']['test'].insert_many(documents)

In [11]:
from corenlp_pywrap import pywrap
full_annotator_list = ["ssplit"]
cn = pywrap.CoreNLP(url='http://172.17.0.2:9000', annotator_list = full_annotator_list)

In [12]:
re.split(r'([\n\\n]*[ A-Z,-{(}{)}]+[:])', video_clips[2])

['\n ',
 '\nJOHN KELLY, HOMELAND SECURITY SECRETARY:',
 ' In retrospect, I should have -- this is all on me, by the way -- I should have delayed it just a bit so that I could talk to members of Congress, particularly the leadership of committees like this, to prepare them for what was coming. \n \nThe desire was to get it out. The thinking was to get it out quick so that potentially people that might be coming here to harm us would not take advantage. \n \n']

In [13]:
def clean_speaker(speaker):
    out = re.sub(r'[^a-zA-Z,. ]+', '', speaker)
    return out.strip()
def clean_text(text):
    out = re.sub(r'[\\]+t|[\\]+n', '', text)
    out = re.sub(r"\n|\t|[\\]+|<[/]*p>|\([A-Z ]*\)|\\\\t", '', out)
    out = re.sub(r'\'', "'", out)
    return str(out)

In [346]:
re.split(r'([\n\\n]*[ A-Z,-,{(}{)}]+[:])', "\nUNINDITIFIED MALE:  Who do you want to eat more kale in Washington? RUTH BADER GINSBURG, JUSTICE, SUPREME COURT:")

['',
 '\nUNINDITIFIED MALE:',
 '  Who do you want to eat more kale in Washington?',
 ' RUTH BADER GINSBURG, JUSTICE, SUPREME COURT:',
 '']

In [262]:
[re.match(r'([\n\\n]*[ A-Z,-{(}{)}]+[:])', i) for i in x]

[None,
 <_sre.SRE_Match object; span=(0, 23), match=':JOHN KING, CNN ANCHOR:'>,
 None]

In [211]:
doc['text']

KeyError: 'text'

In [226]:
string = "s medical deferment from the military.  During a radio interview, the retired general was asked a lighthearted question"

In [231]:
re.match(r'([\n\\n]*[ A-Z,-]+:)', '\n Well:')

In [29]:
sample = c['corpora']['cnn_processed'].aggregate([
    {'$sample': {'size':5000}}
])

In [30]:
total = 0
for x in sample:
    total += len(x['convo'])
total / 5000.

26.4718